# MATH 578 Lab Assignment 1 <a class="tocSkip">


***Kai Yang***

# PART 1
**In this exercise, we will do an experimental study of the growth factor for Gaussian elimination with partial pivoting. Given the factorization $PA = LU$, deﬁne the growth factor by 
\[
g\left(A\right)=\frac{\max_{i,j}\left|u_{ij}\right|}{\max_{i,j}\left|a_{ij}\right|}
\]
where $u_{ij}$ and $a_{ij}$ are the elements of $U$ and $A$, respectively.** 

## Problem 1
**A relevant paper is L.N. Trefethen and R.S. Schreiber. Average-case stability of Gaussian elimination. Please read this paper in suﬃcient detail to understand the main points.**

## Problem 2 
**Write a function that takes $n$ as its parameter and generates an $n \times n$ matrix, whose entries are random numbers with a normal distribution. Pick the parameters ($\mu$ and $\sigma$) at your convenience.**

## Problem 3
**Implement Gaussian elimination with partial pivoting, and test it on a number of cases to be sure of its correctness.**

## Problem 4
**Plot the growth factor against the matrix size, in logarithmic scale, where the matrices are generated as in Step 2. The sample of matrices should be large enough to be a good representative of random matrices with $n$ ranging between, say $10$ and $1000$ (larger sizes are welcome, if it does not take too long). In particular, for each $n$, one should experiment on a generous number of matrices. From the experimental plot, estimate the power $\alpha$ in the assumed dependence $g\approx cn^\alpha$, where $g$ is the growth factor, $n$ is the matrix size, and $c$ is a constant. Compare this with the worst case scenario $g\approx 2^n$.**

## Problem 5
**Now we study the probability distribution of the growth factor for a ﬁxed $n$. Fix $n$, say $n = 10$ or $n = 16$, and generate an abundant number of random matrices, as in Step 2, to measure their growth factors. Then by subdividing the value-space of the growth factors into small subintervals of equal length, and by counting the number of matrices with growth factor lying in each of those subintervals, produce an approximation of the probability density function of the growth factor (the usual ”histogram” technique). Plot it against the growth factor value, with the vertical axis in logarithmic scale. Make a conjecture on how the probability density decays as the growth factor becomes large. Note that the number of matrices and the length of the subintervals should be so that most of the subintervals individually contain a large number of matrices, and that there are enough subintervals to give a meaningful approximation of the probability density function (i.e., the width of bars in the histogram must be small, but most of the bars must still include a large number of cases). Repeat the experiment for several values of $n$, say $n = 20, 40, 80$, or $n = 32, 64, 128$.** 

# PART 2
**Let $A$ be the $1000\times 1000$ tridiagonal symmetric matrix with $1, 2, . . . , 1000$ on the diagonal and $1$ on the sub- and super-diagonals, and let $b = \left(1, 1,\dots , 1\right)^T$ .**

## Problem 1
**Write a code that performs $1000$ (or more) steps of the steepest descent iteration to approximately solve $Ax = b$.**

## Problem 2
**Write a code that performs $1000$ steps of the CG to approximately solve $Ax = b$.**

## Problem 3
**Produce a plot with four curves on it:**

- **the computed residual norms $\left\Vert r_{i}\right\Vert _{2}$ for CG,**
- **the actual residual norms $\left\Vert b-Ax_{i}\right\Vert _{2}$ for CG,**
- **the residual norms $\left\Vert r_{i}\right\Vert _{2}$ for steepest descent, and**
- **the estimate $\frac{2\left(\sqrt{\kappa}-1\right)^{i}}{\left(\sqrt{\kappa}+1\right)^{i}}$, where $\kappa=\left\Vert A\right\Vert \left\Vert A^{-1}\right\Vert $ is the $2$-norm condition number of $A$ (which can be estimated by `NumPy` function `linalg.cond`), and $i$ is the iteration number that is to be on the horizontal axis of the plot.**

**Choose logarithmic scale for the vertical axis if appropriate.**

## Problem 4
**Comment on your results.**